In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
aqsol = pd.read_csv('../AqSolDB/aqsoldb_clean_1115.csv')
aqsol.head()

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences
0,1,A,3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1
1,2,A,4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1
2,3,A,5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1
3,4,A,8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1
4,5,A,9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1


In [3]:
# henry = pd.read_csv('../HLCDB/henry_clean.csv')
henry = pd.read_csv('../HLCDB/henry_shrinked.csv')
henry.head()

,id,avg_kh,med_kh,std_kh,species_id,occurrences
0,1.0,0.000013,0.000013,4.522670e-07,1.0,12.0
1,2.0,0.000110,0.000110,1.294815e-05,2.0,11.0
2,3.0,0.000003,0.000003,0.000000e+00,3.0,2.0
3,4.0,0.000008,0.000008,8.944272e-08,4.0,5.0
4,5.0,10.351111,0.320000,2.949868e+01,6.0,9.0


In [4]:
print(aqsol.shape, henry.shape)

(9982, 10) (4513, 6)


In [5]:
# calculate avg/geom mean/median for KH value and std deviation to see which one's best for ML

In [6]:
reference = pd.read_csv('Alastair_MergedDataset.csv')

In [7]:
reference.shape

(882, 218)

In [59]:
# species = pd.read_csv('../HLCDB/species.csv', on_bad_lines='skip') # error  on two lines: Error tokenizing data. C error: Expected 7 fields in line 1765, saw 8
# species = pd.read_csv('../HLCDB/species.csv')
species = pd.read_csv('../HLCDB/species_excelled.csv').drop(columns=['Unnamed: 7'])
species

,id,iupac,formula,trivial,casrn,inchikey,subcat_id
0,1,oxygen,O<sub>2</sub>,NaN,7782-44-7,MYMOFIZGZYHOMD-UHFFFAOYSA-N,1
1,2,ozone,O<sub>3</sub>,NaN,10028-15-6,CBENFWSGALASAD-UHFFFAOYSA-N,1
2,3,hydrogen atom,H,NaN,12385-13-6,YZCKVEUIGOORGS-UHFFFAOYSA-N,2
3,4,hydrogen,H<sub>2</sub>,NaN,1333-74-0,UFHFLCQGNIYNRP-UHFFFAOYSA-N,2
4,5,deuterium,D<sub>2</sub>,NaN,7782-39-0,UFHFLCQGNIYNRP-VVKOMZTBSA-N,2
...,...,...,...,...,...,...,...
4627,4628,tetramethyl lead,C<sub>4</sub>H<sub>12</sub>Pb,NaN,75-74-1,XOOGZRUBTYCLHG-UHFFFAOYSA-N,61
4628,4629,ethyltrimethylplumbane,C<sub>5</sub>H<sub>14</sub>Pb,NaN,1762-26-1,KHQJREYATBQBHY-UHFFFAOYSA-N,61
4629,4630,diethyldimethylplumbane,C<sub>6</sub>H<sub>16</sub>Pb,diethyldimethyl lead,1762-27-2,OLOAJSHVLXNSQV-UHFFFAOYSA-N,61
4630,4631,triethylmethylplumbane,C<sub>7</sub>H<sub>18</sub>Pb,methyltriethyl lead,1762-28-3,KGFRUGHBHNUHOS-UHFFFAOYSA-N,61


In [9]:
print(species.formula[1380])
print(species.formula[1380].replace('<sub>','').replace('</sub>','_'))

C<sub>20</sub>H<sub>34</sub>O<sub>4</sub>
C20_H34_O4_


In [10]:
henry.merge(species, left_on='species_id', right_on='id')

,id_x,avg_kh,med_kh,std_kh,species_id,occurrences,id_y,iupac,formula,trivial,casrn,inchikey,subcat_id,Unnamed: 7
0,1.0,0.000013,0.000013,4.522670e-07,1.0,12.0,1,oxygen,O<sub>2</sub>,NaN,7782-44-7,MYMOFIZGZYHOMD-UHFFFAOYSA-N,1,NaN
1,2.0,0.000110,0.000110,1.294815e-05,2.0,11.0,2,ozone,O<sub>3</sub>,NaN,10028-15-6,CBENFWSGALASAD-UHFFFAOYSA-N,1,NaN
2,3.0,0.000003,0.000003,0.000000e+00,3.0,2.0,3,hydrogen atom,H,NaN,12385-13-6,YZCKVEUIGOORGS-UHFFFAOYSA-N,2,NaN
3,4.0,0.000008,0.000008,8.944272e-08,4.0,5.0,4,hydrogen,H<sub>2</sub>,NaN,1333-74-0,UFHFLCQGNIYNRP-UHFFFAOYSA-N,2,NaN
4,5.0,10.351111,0.320000,2.949868e+01,6.0,9.0,6,hydroxyl radical,OH,NaN,3352-57-6,TUJKJAMUKRIRHC-UHFFFAOYSA-N,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4508,4509.0,0.000016,0.000016,0.000000e+00,4628.0,1.0,4628,tetramethyl lead,C<sub>4</sub>H<sub>12</sub>Pb,NaN,75-74-1,XOOGZRUBTYCLHG-UHFFFAOYSA-N,61,NaN
4509,4510.0,0.000028,0.000028,0.000000e+00,4629.0,1.0,4629,ethyltrimethylplumbane,C<sub>5</sub>H<sub>14</sub>Pb,NaN,1762-26-1,KHQJREYATBQBHY-UHFFFAOYSA-N,61,NaN
4510,4511.0,0.000021,0.000021,0.000000e+00,4630.0,1.0,4630,diethyldimethylplumbane,C<sub>6</sub>H<sub>16</sub>Pb,diethyldimethyl lead,1762-27-2,OLOAJSHVLXNSQV-UHFFFAOYSA-N,61,NaN
4511,4512.0,0.000016,0.000016,0.000000e+00,4631.0,1.0,4631,triethylmethylplumbane,C<sub>7</sub>H<sub>18</sub>Pb,methyltriethyl lead,1762-28-3,KGFRUGHBHNUHOS-UHFFFAOYSA-N,61,NaN


In [11]:
# henry.merge(species, left_on='species_id', right_on='id')

In [12]:
species.head()

,id,iupac,formula,trivial,casrn,inchikey,subcat_id,Unnamed: 7
0,1,oxygen,O<sub>2</sub>,NaN,7782-44-7,MYMOFIZGZYHOMD-UHFFFAOYSA-N,1,NaN
1,2,ozone,O<sub>3</sub>,NaN,10028-15-6,CBENFWSGALASAD-UHFFFAOYSA-N,1,NaN
2,3,hydrogen atom,H,NaN,12385-13-6,YZCKVEUIGOORGS-UHFFFAOYSA-N,2,NaN
3,4,hydrogen,H<sub>2</sub>,NaN,1333-74-0,UFHFLCQGNIYNRP-UHFFFAOYSA-N,2,NaN
4,5,deuterium,D<sub>2</sub>,NaN,7782-39-0,UFHFLCQGNIYNRP-VVKOMZTBSA-N,2,NaN


In [13]:
species.merge(aqsol, how='inner',left_on='inchikey', right_on='inchikey')

,id_x,iupac,formula,trivial,casrn,inchikey,subcat_id,Unnamed: 7,id_y,dataset,setnumber,name,inchi,smiles,solubility,sd,ocurrences
0,1,oxygen,O<sub>2</sub>,NaN,7782-44-7,MYMOFIZGZYHOMD-UHFFFAOYSA-N,1,NaN,5469,B,2334,oxygen,InChI=1S/O2/c1-2,O=O,0.069100,0.000000,1
1,8,hydrogen peroxide,H<sub>2</sub>O<sub>2</sub>,NaN,7722-84-1,MHAJPDPJQMAIIY-UHFFFAOYSA-N,2,NaN,1112,A,1902,peroxol,InChI=1S/H2O2/c1-2/h1-2H,OO,0.468342,0.000000,1
2,9,nitrogen,N<sub>2</sub>,NaN,7727-37-9,IJGRMHOSHXDMSA-UHFFFAOYSA-N,3,NaN,5466,B,2331,nitrogen,InChI=1S/N2/c1-2,N#N,-0.190100,0.000000,1
3,10,ammonia,NH<sub>3</sub>,NaN,7664-41-7,QGZKDVFQNNGYKY-UHFFFAOYSA-N,3,NaN,420,A,719,ammonia,InChI=1S/H3N/h1H3,N,1.451807,0.029165,4
4,12,hydrazine,H<sub>4</sub>N<sub>2</sub>,NaN,302-01-2,OAKJQQAXSVQMHS-UHFFFAOYSA-N,3,NaN,4228,B,894,hydrazine,InChI=1S/H4N2/c1-2/h1-2H2,NN,1.494200,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401,4601,tetrabutylstannane,C<sub>16</sub>H<sub>36</sub>Sn,tetra-butyl tin,1461-25-2,AFCAKJKUYFLYFK-UHFFFAOYSA-N,59,NaN,198,A,339,tetrabutylstannane,"InChI=1S/4C4H9.Sn/c4*1-3-4-2;/h4*1,3-4H2,2H3;",CCCC[Sn](CCCC)(CCCC)CCCC,-4.596066,0.980203,2
2402,4602,hexabutyldistannoxane,C<sub>24</sub>H<sub>54</sub>OSn<sub>2</sub>,bis(tributyltin)oxide,56-35-9,APQHKWPGGHMYKJ-UHFFFAOYSA-N,59,NaN,238,A,398,tributyl[(tributylstannyl)oxy]stannane,"InChI=1S/6C4H9.O.2Sn/c6*1-3-4-2;;;/h6*1,3-4H2,...",CCCC[Sn](CCCC)(CCCC)O[Sn](CCCC)(CCCC)CCCC,-3.922852,0.000000,1
2403,4612,phenyl mercuric ethanoate,C<sub>8</sub>H<sub>8</sub>HgO<sub>2</sub>,NaN,62-38-4,XEBWQGVWTUSTLN-UHFFFAOYSA-M,60,NaN,7016,B,4351,phenylmercuric acetate,InChI=1S/C6H5.C2H4O2.Hg/c1-2-4-6-5-3-1;1-2(3)4...,CC(=O)O[Hg]c1ccccc1,-1.886800,0.169600,2
2404,4621,2-methoxyethylmercury chloride,CH<sub>3</sub>OC<sub>2</sub>H<sub>4</sub>HgCl,aretan,123-88-6,VJTAZCKMHINUKO-UHFFFAOYSA-M,60,NaN,7017,B,4352,2-methoxyethyl mercuric chloride,"InChI=1S/C3H7O.ClH.Hg/c1-3-4-2;;/h1,3H2,2H3;1H...",[Cl-].COCC[Hg+],-0.771000,0.000000,1


In [14]:
# 2406 cannot be as there are repating compounds in aqsol
# get unique inchikeys from aqsol and compare

In [15]:
inchikey_hlc = species['inchikey']

In [16]:
sum(aqsol['inchikey'].value_counts() > 1)

0

In [17]:
inchikey_aqsol = aqsol['inchikey']
inchikey_aqsol
# do not confuse id w shape

0       SZEMGTQCPRNXEG-UHFFFAOYSA-M
1       GPYLCFQEKPUWLD-UHFFFAOYSA-N
2       AVPYQKSLYISFPO-UHFFFAOYSA-N
3       XTUPUYCJWKHGSW-UHFFFAOYSA-L
4       FAUAZXVRLVIARB-UHFFFAOYSA-N
                   ...             
9977    GKCBAIGFKIBETG-UHFFFAOYSA-N
9978    OFVLGDICTFRJMM-WESIUVDSSA-N
9979    MGSRCZKZVOBKFT-UHFFFAOYSA-N
9980    SGTNSNPWRIOYBX-UHFFFAOYSA-N
9981    PJVWKTKQMONHTI-UHFFFAOYSA-N
Name: inchikey, Length: 9982, dtype: object

In [18]:
# checking uniqueness
print(inchikey_aqsol.shape)
inchikey_aqsol.value_counts()#.sum()

(9982,)


SZEMGTQCPRNXEG-UHFFFAOYSA-M    1
OBETXYAYXDNJHR-UHFFFAOYSA-N    1
OXQGTIUCKGYOAA-UHFFFAOYSA-N    1
WHKUVVPPKQRRBV-UHFFFAOYSA-N    1
WLJVNTCWHIRURA-UHFFFAOYSA-N    1
                              ..
SIFKXZAVMBSRMB-UHFFFAOYSA-N    1
OXEFCDNMUKTKDV-UHFFFAOYSA-N    1
HJICBVMRKHYRDH-UHFFFAOYSA-N    1
PNVNVHUZROJLTJ-UHFFFAOYSA-N    1
PJVWKTKQMONHTI-UHFFFAOYSA-N    1
Name: inchikey, Length: 9982, dtype: int64

In [19]:
common_inchikey = set(inchikey_hlc) & set(inchikey_aqsol)
print(len(common_inchikey))

2406


In [20]:
print(len(set(inchikey_hlc)))
print(len(set(species['inchikey'])))
print(species.shape)

4426
4426
(4632, 8)


In [21]:
species.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4632 entries, 0 to 4631
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          4632 non-null   int64  
 1   iupac       4632 non-null   object 
 2   formula     4632 non-null   object 
 3   trivial     1577 non-null   object 
 4   casrn       4631 non-null   object 
 5   inchikey    4632 non-null   object 
 6   subcat_id   4632 non-null   object 
 7   Unnamed: 7  2 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 289.6+ KB


In [22]:
print(len(set(inchikey_aqsol)))
print(len(set(aqsol['inchikey'])))

9982
9982


In [23]:
species.shape

(4632, 8)

In [24]:
# aqsol

In [25]:

species.merge(inchikey_aqsol,how='inner',left_on='inchikey',right_on='inchikey')#['inchikey'].value_counts()

,id,iupac,formula,trivial,casrn,inchikey,subcat_id,Unnamed: 7
0,1,oxygen,O<sub>2</sub>,NaN,7782-44-7,MYMOFIZGZYHOMD-UHFFFAOYSA-N,1,NaN
1,8,hydrogen peroxide,H<sub>2</sub>O<sub>2</sub>,NaN,7722-84-1,MHAJPDPJQMAIIY-UHFFFAOYSA-N,2,NaN
2,9,nitrogen,N<sub>2</sub>,NaN,7727-37-9,IJGRMHOSHXDMSA-UHFFFAOYSA-N,3,NaN
3,10,ammonia,NH<sub>3</sub>,NaN,7664-41-7,QGZKDVFQNNGYKY-UHFFFAOYSA-N,3,NaN
4,12,hydrazine,H<sub>4</sub>N<sub>2</sub>,NaN,302-01-2,OAKJQQAXSVQMHS-UHFFFAOYSA-N,3,NaN
...,...,...,...,...,...,...,...,...
2401,4601,tetrabutylstannane,C<sub>16</sub>H<sub>36</sub>Sn,tetra-butyl tin,1461-25-2,AFCAKJKUYFLYFK-UHFFFAOYSA-N,59,NaN
2402,4602,hexabutyldistannoxane,C<sub>24</sub>H<sub>54</sub>OSn<sub>2</sub>,bis(tributyltin)oxide,56-35-9,APQHKWPGGHMYKJ-UHFFFAOYSA-N,59,NaN
2403,4612,phenyl mercuric ethanoate,C<sub>8</sub>H<sub>8</sub>HgO<sub>2</sub>,NaN,62-38-4,XEBWQGVWTUSTLN-UHFFFAOYSA-M,60,NaN
2404,4621,2-methoxyethylmercury chloride,CH<sub>3</sub>OC<sub>2</sub>H<sub>4</sub>HgCl,aretan,123-88-6,VJTAZCKMHINUKO-UHFFFAOYSA-M,60,NaN


In [26]:
# henry.species table has unique entries
# so there are 1310 species where both HLC and aqsol data is available?

In [27]:
henry.head()

,id,avg_kh,med_kh,std_kh,species_id,occurrences
0,1.0,0.000013,0.000013,4.522670e-07,1.0,12.0
1,2.0,0.000110,0.000110,1.294815e-05,2.0,11.0
2,3.0,0.000003,0.000003,0.000000e+00,3.0,2.0
3,4.0,0.000008,0.000008,8.944272e-08,4.0,5.0
4,5.0,10.351111,0.320000,2.949868e+01,6.0,9.0


In [28]:
henry[['id','avg_kh','species_id','occurrences']].merge(species[['id','inchikey']], how='inner', left_on='species_id', right_on='id')

,id_x,avg_kh,species_id,occurrences,id_y,inchikey
0,1.0,0.000013,1.0,12.0,1,MYMOFIZGZYHOMD-UHFFFAOYSA-N
1,2.0,0.000110,2.0,11.0,2,CBENFWSGALASAD-UHFFFAOYSA-N
2,3.0,0.000003,3.0,2.0,3,YZCKVEUIGOORGS-UHFFFAOYSA-N
3,4.0,0.000008,4.0,5.0,4,UFHFLCQGNIYNRP-UHFFFAOYSA-N
4,5.0,10.351111,6.0,9.0,6,TUJKJAMUKRIRHC-UHFFFAOYSA-N
...,...,...,...,...,...,...
4508,4509.0,0.000016,4628.0,1.0,4628,XOOGZRUBTYCLHG-UHFFFAOYSA-N
4509,4510.0,0.000028,4629.0,1.0,4629,KHQJREYATBQBHY-UHFFFAOYSA-N
4510,4511.0,0.000021,4630.0,1.0,4630,OLOAJSHVLXNSQV-UHFFFAOYSA-N
4511,4512.0,0.000016,4631.0,1.0,4631,KGFRUGHBHNUHOS-UHFFFAOYSA-N


In [29]:
aqsol.head()

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences
0,1,A,3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1
1,2,A,4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1
2,3,A,5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1
3,4,A,8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1
4,5,A,9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1


In [30]:
henry[['id','avg_kh','species_id','occurrences']].merge(species[['id','inchikey']], how='inner', left_on='species_id', right_on='id'
    ).merge(aqsol,how='inner',left_on='inchikey',right_on='inchikey')

,id_x,avg_kh,species_id,occurrences,id_y,inchikey,id,dataset,setnumber,name,inchi,smiles,solubility,sd,ocurrences
0,1.0,0.000013,1.0,12.0,1,MYMOFIZGZYHOMD-UHFFFAOYSA-N,5469,B,2334,oxygen,InChI=1S/O2/c1-2,O=O,0.069100,0.000000,1
1,7.0,828.933333,8.0,15.0,8,MHAJPDPJQMAIIY-UHFFFAOYSA-N,1112,A,1902,peroxol,InChI=1S/H2O2/c1-2/h1-2H,OO,0.468342,0.000000,1
2,8.0,0.000006,9.0,10.0,9,IJGRMHOSHXDMSA-UHFFFAOYSA-N,5466,B,2331,nitrogen,InChI=1S/N2/c1-2,N#N,-0.190100,0.000000,1
3,9.0,0.534118,10.0,17.0,10,QGZKDVFQNNGYKY-UHFFFAOYSA-N,420,A,719,ammonia,InChI=1S/H3N/h1H3,N,1.451807,0.029165,4
4,11.0,16.000000,12.0,1.0,12,OAKJQQAXSVQMHS-UHFFFAOYSA-N,4228,B,894,hydrazine,InChI=1S/H4N2/c1-2/h1-2H2,NN,1.494200,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,4482.0,0.000002,4601.0,1.0,4601,AFCAKJKUYFLYFK-UHFFFAOYSA-N,198,A,339,tetrabutylstannane,"InChI=1S/4C4H9.Sn/c4*1-3-4-2;/h4*1,3-4H2,2H3;",CCCC[Sn](CCCC)(CCCC)CCCC,-4.596066,0.980203,2
2360,4483.0,76.000000,4602.0,1.0,4602,APQHKWPGGHMYKJ-UHFFFAOYSA-N,238,A,398,tributyl[(tributylstannyl)oxy]stannane,"InChI=1S/6C4H9.O.2Sn/c6*1-3-4-2;;;/h6*1,3-4H2,...",CCCC[Sn](CCCC)(CCCC)O[Sn](CCCC)(CCCC)CCCC,-3.922852,0.000000,1
2361,4493.0,15000.000000,4612.0,1.0,4612,XEBWQGVWTUSTLN-UHFFFAOYSA-M,7016,B,4351,phenylmercuric acetate,InChI=1S/C6H5.C2H4O2.Hg/c1-2-4-6-5-3-1;1-2(3)4...,CC(=O)O[Hg]c1ccccc1,-1.886800,0.169600,2
2362,4502.0,3900.000000,4621.0,1.0,4621,VJTAZCKMHINUKO-UHFFFAOYSA-M,7017,B,4352,2-methoxyethyl mercuric chloride,"InChI=1S/C3H7O.ClH.Hg/c1-3-4-2;;/h1,3H2,2H3;1H...",[Cl-].COCC[Hg+],-0.771000,0.000000,1


In [31]:
len(common_inchikey)

2406

In [32]:
species[['id','inchikey']]

,id,inchikey
0,1,MYMOFIZGZYHOMD-UHFFFAOYSA-N
1,2,CBENFWSGALASAD-UHFFFAOYSA-N
2,3,YZCKVEUIGOORGS-UHFFFAOYSA-N
3,4,UFHFLCQGNIYNRP-UHFFFAOYSA-N
4,5,UFHFLCQGNIYNRP-VVKOMZTBSA-N
...,...,...
4627,4628,XOOGZRUBTYCLHG-UHFFFAOYSA-N
4628,4629,KHQJREYATBQBHY-UHFFFAOYSA-N
4629,4630,OLOAJSHVLXNSQV-UHFFFAOYSA-N
4630,4631,KGFRUGHBHNUHOS-UHFFFAOYSA-N


In [33]:
henry['inchikey'] = henry['id'].apply(lambda x: species['inchikey'][x])
henry

,id,avg_kh,med_kh,std_kh,species_id,occurrences,inchikey
0,1.0,0.000013,0.000013,4.522670e-07,1.0,12.0,CBENFWSGALASAD-UHFFFAOYSA-N
1,2.0,0.000110,0.000110,1.294815e-05,2.0,11.0,YZCKVEUIGOORGS-UHFFFAOYSA-N
2,3.0,0.000003,0.000003,0.000000e+00,3.0,2.0,UFHFLCQGNIYNRP-UHFFFAOYSA-N
3,4.0,0.000008,0.000008,8.944272e-08,4.0,5.0,UFHFLCQGNIYNRP-VVKOMZTBSA-N
4,5.0,10.351111,0.320000,2.949868e+01,6.0,9.0,TUJKJAMUKRIRHC-UHFFFAOYSA-N
...,...,...,...,...,...,...,...
4508,4509.0,0.000016,0.000016,0.000000e+00,4628.0,1.0,LESVOLZBIFDZGS-UHFFFAOYSA-N
4509,4510.0,0.000028,0.000028,0.000000e+00,4629.0,1.0,SCKHCCSZFPSHGR-UHFFFAOYSA-N
4510,4511.0,0.000021,0.000021,0.000000e+00,4630.0,1.0,ZNOLGFHPUIJIMJ-UHFFFAOYSA-N
4511,4512.0,0.000016,0.000016,0.000000e+00,4631.0,1.0,DUFVKSUJRWYZQP-UHFFFAOYSA-N


In [34]:
aqsol.columns

Index(['id', 'dataset', 'setnumber', 'name', 'inchi', 'inchikey', 'smiles',
       'solubility', 'sd', 'ocurrences'],
      dtype='object')

In [35]:
merged = aqsol[['solubility','sd','ocurrences','inchikey']].merge(henry[['avg_kh','med_kh','std_kh','occurrences','inchikey','species_id']], how='inner')
merged

,solubility,sd,ocurrences,inchikey,avg_kh,med_kh,std_kh,occurrences,species_id
0,-3.123150,0.000000,1,JZHGRUMIRATHIU-UHFFFAOYSA-N,0.000382,0.000370,0.000115,4.0,426.0
1,-2.466031,0.060621,4,WNTGYJSOUMFZEP-UHFFFAOYSA-N,0.023000,0.023000,0.000000,1.0,3140.0
2,-3.411514,0.000000,1,YTCGOUNVIAWCMG-UHFFFAOYSA-N,0.025800,0.022000,0.026505,3.0,3798.0
3,-4.287343,0.131231,3,MOYAFQVGZZPNRA-UHFFFAOYSA-N,0.000455,0.000455,0.000488,2.0,441.0
4,-4.292313,1.247787,3,WTARULDDTDQWMU-UHFFFAOYSA-N,0.000480,0.000380,0.000149,5.0,433.0
...,...,...,...,...,...,...,...,...,...
2336,-3.870000,0.790000,2,XQYZDYMELSJDRZ-UHFFFAOYSA-N,0.310000,0.310000,0.000000,1.0,2249.0
2337,-2.290000,0.095670,5,DDBREPKUVSBGFI-UHFFFAOYSA-N,0.006100,0.006400,0.000751,7.0,2213.0
2338,-2.730000,0.230750,2,ASWVTGNCAZCNNR-UHFFFAOYSA-N,0.930000,0.930000,0.000000,1.0,4302.0
2339,-2.190000,0.019222,3,MGSRCZKZVOBKFT-UHFFFAOYSA-N,34.666667,42.000000,18.147543,3.0,835.0


In [36]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2341 entries, 0 to 2340
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   solubility   2341 non-null   float64
 1   sd           2341 non-null   float64
 2   ocurrences   2341 non-null   int64  
 3   inchikey     2341 non-null   object 
 4   avg_kh       2341 non-null   float64
 5   med_kh       2341 non-null   float64
 6   std_kh       2341 non-null   float64
 7   occurrences  2341 non-null   float64
 8   species_id   2341 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 182.9+ KB


In [37]:
# now let's get compounds from aqsol that are not in merged -> extend species table with it innit
# extend aqsol table with full species (existing and new)

In [38]:
new_aqsol = aqsol.copy(deep=True)
new_species = species.copy(deep=True)

In [39]:
id_spec_max = species['id'].max()
id_spec_max
# for the aqsol compounds that will be added, counting should start from here

4632

In [40]:
test = 'MYMOFIZGZYHOMD-UHFFFAOYSA-N'
species[species['inchikey']==test]['id']

0    1
Name: id, dtype: int64

In [41]:
matched_aqsol = aqsol.loc[aqsol['inchikey'].isin(merged['inchikey'])].copy(deep=True)
matched_aqsol
# add existing species id to these

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences
5,6,A,10,vinyltoluene,"InChI=1S/C9H10/c1-3-9-6-4-5-8(2)7-9/h3-7H,1H2,2H3",JZHGRUMIRATHIU-UHFFFAOYSA-N,Cc1cccc(C=C)c1,-3.123150,0.000000,1
16,17,A,24,2-(4-chloro-2-methylphenoxy)propanoic acid,InChI=1S/C10H11ClO3/c1-6-5-8(11)3-4-9(6)14-7(2...,WNTGYJSOUMFZEP-UHFFFAOYSA-N,CC(Oc1ccc(Cl)cc1C)C(O)=O,-2.466031,0.060621,4
18,19,A,26,1-chloro-3-(trifluoromethyl)benzene,"InChI=1S/C7H4ClF3/c8-6-3-1-2-5(4-6)7(9,10)11/h...",YTCGOUNVIAWCMG-UHFFFAOYSA-N,FC(F)(F)c1cccc(Cl)c1,-3.411514,0.000000,1
21,22,A,29,1-methyl-4-(propan-2-ylidene)cyclohex-1-ene,"InChI=1S/C10H16/c1-8(2)10-6-4-9(3)5-7-10/h4H,5...",MOYAFQVGZZPNRA-UHFFFAOYSA-N,CC(C)=C1CCC(=CC1)C,-4.287343,0.131231,3
27,28,A,36,"(1S,5S)-6,6-dimethyl-2-methylidenebicyclo[3.1....","InChI=1S/C10H16/c1-7-4-5-8-6-9(7)10(8,2)3/h8-9...",WTARULDDTDQWMU-UHFFFAOYSA-N,CC1(C)C2CCC(=C)C1C2,-4.292313,1.247787,3
...,...,...,...,...,...,...,...,...,...,...
9965,9966,I,63,papaverine,InChI=1S/C20H21NO4/c1-22-17-6-5-13(10-18(17)23...,XQYZDYMELSJDRZ-UHFFFAOYSA-N,c1(nccc2cc(c(OC)cc12)OC)Cc1cc(c(cc1)OC)OC,-3.870000,0.790000,2
9967,9968,I,66,phenobarbital,InChI=1S/C12H12N2O3/c1-2-12(8-6-4-3-5-7-8)9(15...,DDBREPKUVSBGFI-UHFFFAOYSA-N,C1(NC(C(c2ccccc2)(C(=O)N1)CC)=O)=O,-2.290000,0.095670,5
9974,9975,I,79,sulfamethazine,InChI=1S/C12H14N4O2S/c1-8-7-9(2)15-12(14-8)16-...,ASWVTGNCAZCNNR-UHFFFAOYSA-N,S(=O)(=O)(Nc1nc(C)cc(n1)C)c1ccc(N)cc1,-2.730000,0.230750,2
9979,9980,I,86,thymol,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)6-10(9)11/h4...,MGSRCZKZVOBKFT-UHFFFAOYSA-N,c1(cc(ccc1C(C)C)C)O,-2.190000,0.019222,3


In [42]:
species.loc[species['inchikey'].isin(matched_aqsol['inchikey'])][['id','inchikey']]

,id,inchikey
7,8,MHAJPDPJQMAIIY-UHFFFAOYSA-N
8,9,IJGRMHOSHXDMSA-UHFFFAOYSA-N
9,10,QGZKDVFQNNGYKY-UHFFFAOYSA-N
11,12,OAKJQQAXSVQMHS-UHFFFAOYSA-N
12,13,GQPLMRYTRLFLPF-UHFFFAOYSA-N
...,...,...
4509,4509,LESVOLZBIFDZGS-UHFFFAOYSA-N
4510,4510,SCKHCCSZFPSHGR-UHFFFAOYSA-N
4511,4511,ZNOLGFHPUIJIMJ-UHFFFAOYSA-N
4512,4512,DUFVKSUJRWYZQP-UHFFFAOYSA-N


In [43]:
matched_aqsol['species_id'] = np.nan
matched_aqsol

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences,species_id
5,6,A,10,vinyltoluene,"InChI=1S/C9H10/c1-3-9-6-4-5-8(2)7-9/h3-7H,1H2,2H3",JZHGRUMIRATHIU-UHFFFAOYSA-N,Cc1cccc(C=C)c1,-3.123150,0.000000,1,NaN
16,17,A,24,2-(4-chloro-2-methylphenoxy)propanoic acid,InChI=1S/C10H11ClO3/c1-6-5-8(11)3-4-9(6)14-7(2...,WNTGYJSOUMFZEP-UHFFFAOYSA-N,CC(Oc1ccc(Cl)cc1C)C(O)=O,-2.466031,0.060621,4,NaN
18,19,A,26,1-chloro-3-(trifluoromethyl)benzene,"InChI=1S/C7H4ClF3/c8-6-3-1-2-5(4-6)7(9,10)11/h...",YTCGOUNVIAWCMG-UHFFFAOYSA-N,FC(F)(F)c1cccc(Cl)c1,-3.411514,0.000000,1,NaN
21,22,A,29,1-methyl-4-(propan-2-ylidene)cyclohex-1-ene,"InChI=1S/C10H16/c1-8(2)10-6-4-9(3)5-7-10/h4H,5...",MOYAFQVGZZPNRA-UHFFFAOYSA-N,CC(C)=C1CCC(=CC1)C,-4.287343,0.131231,3,NaN
27,28,A,36,"(1S,5S)-6,6-dimethyl-2-methylidenebicyclo[3.1....","InChI=1S/C10H16/c1-7-4-5-8-6-9(7)10(8,2)3/h8-9...",WTARULDDTDQWMU-UHFFFAOYSA-N,CC1(C)C2CCC(=C)C1C2,-4.292313,1.247787,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9965,9966,I,63,papaverine,InChI=1S/C20H21NO4/c1-22-17-6-5-13(10-18(17)23...,XQYZDYMELSJDRZ-UHFFFAOYSA-N,c1(nccc2cc(c(OC)cc12)OC)Cc1cc(c(cc1)OC)OC,-3.870000,0.790000,2,NaN
9967,9968,I,66,phenobarbital,InChI=1S/C12H12N2O3/c1-2-12(8-6-4-3-5-7-8)9(15...,DDBREPKUVSBGFI-UHFFFAOYSA-N,C1(NC(C(c2ccccc2)(C(=O)N1)CC)=O)=O,-2.290000,0.095670,5,NaN
9974,9975,I,79,sulfamethazine,InChI=1S/C12H14N4O2S/c1-8-7-9(2)15-12(14-8)16-...,ASWVTGNCAZCNNR-UHFFFAOYSA-N,S(=O)(=O)(Nc1nc(C)cc(n1)C)c1ccc(N)cc1,-2.730000,0.230750,2,NaN
9979,9980,I,86,thymol,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)6-10(9)11/h4...,MGSRCZKZVOBKFT-UHFFFAOYSA-N,c1(cc(ccc1C(C)C)C)O,-2.190000,0.019222,3,NaN


In [44]:
ids = []
for c, key in enumerate(matched_aqsol['inchikey']):
    id = int(species[species['inchikey']==key]['id'])
    # print(id)
    ids.append(id)
    # matched_aqsol['species_id'][c] = id

In [45]:
matched_aqsol['species_id'] = ids
matched_aqsol.head()


,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences,species_id
5,6,A,10,vinyltoluene,"InChI=1S/C9H10/c1-3-9-6-4-5-8(2)7-9/h3-7H,1H2,2H3",JZHGRUMIRATHIU-UHFFFAOYSA-N,Cc1cccc(C=C)c1,-3.123150,0.000000,1,418
16,17,A,24,2-(4-chloro-2-methylphenoxy)propanoic acid,InChI=1S/C10H11ClO3/c1-6-5-8(11)3-4-9(6)14-7(2...,WNTGYJSOUMFZEP-UHFFFAOYSA-N,CC(Oc1ccc(Cl)cc1C)C(O)=O,-2.466031,0.060621,4,3058
18,19,A,26,1-chloro-3-(trifluoromethyl)benzene,"InChI=1S/C7H4ClF3/c8-6-3-1-2-5(4-6)7(9,10)11/h...",YTCGOUNVIAWCMG-UHFFFAOYSA-N,FC(F)(F)c1cccc(Cl)c1,-3.411514,0.000000,1,3696
21,22,A,29,1-methyl-4-(propan-2-ylidene)cyclohex-1-ene,"InChI=1S/C10H16/c1-8(2)10-6-4-9(3)5-7-10/h4H,5...",MOYAFQVGZZPNRA-UHFFFAOYSA-N,CC(C)=C1CCC(=CC1)C,-4.287343,0.131231,3,433
27,28,A,36,"(1S,5S)-6,6-dimethyl-2-methylidenebicyclo[3.1....","InChI=1S/C10H16/c1-7-4-5-8-6-9(7)10(8,2)3/h8-9...",WTARULDDTDQWMU-UHFFFAOYSA-N,CC1(C)C2CCC(=C)C1C2,-4.292313,1.247787,3,425


In [46]:
species[species['inchikey']=='WNTGYJSOUMFZEP-UHFFFAOYSA-N']['id']

3058    3058
Name: id, dtype: int64

In [47]:
unmatched_aqsol = aqsol.loc[~aqsol['inchikey'].isin(merged['inchikey'])].copy(deep=True)
unmatched_aqsol
# create new species id for these

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences
0,1,A,3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.00,1
1,2,A,4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.00,1
2,3,A,5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.00,1
3,4,A,8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.00,1
4,5,A,9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.00,1
...,...,...,...,...,...,...,...,...,...,...
9975,9976,I,80,sulfasalazine,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NCEXYHBECQHGNR-QZQOTICOSA-N,C(=O)(O)c1cc(ccc1O)\N=N\c1ccc(cc1)S(=O)(Nc1ccc...,-6.140000,0.00,1
9976,9977,I,83,sulindac_form_II,InChI=1S/C20H17FO3S/c1-12-17(9-13-3-6-15(7-4-1...,MLKXDPUZXIRXEP-RQZCQDPDSA-N,CC1=C(CC(O)=O)c2cc(F)ccc2C\1=C\c1ccc(cc1)S(C)=O,-4.500000,0.41,2
9977,9978,I,84,tetracaine,InChI=1S/C15H24N2O2/c1-4-5-10-16-14-8-6-13(7-9...,GKCBAIGFKIBETG-UHFFFAOYSA-N,C(c1ccc(cc1)NCCCC)(=O)OCCN(C)C,-3.010000,0.00,1
9978,9979,I,85,tetracycline,InChI=1S/C22H24N2O8/c1-21(31)8-5-4-6-11(25)12(...,OFVLGDICTFRJMM-WESIUVDSSA-N,OC1=C(C(C2=C(O)[C@@](C(C(C(N)=O)=C(O)[C@H]3N(C...,-2.930000,0.00,1


In [48]:
unmatched_aqsol['species_id'] = np.nan
unmatched_aqsol.head()

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences,species_id
0,1,A,3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,NaN
1,2,A,4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,NaN
2,3,A,5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,NaN
3,4,A,8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1,NaN
4,5,A,9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,NaN


In [53]:
unmatched_aqsol['species_id'] = np.arange(id_spec_max+1,unmatched_aqsol.shape[0]+id_spec_max+1)

In [54]:
unmatched_aqsol

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences,species_id
0,1,A,3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.00,1,4633
1,2,A,4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.00,1,4634
2,3,A,5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.00,1,4635
3,4,A,8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.00,1,4636
4,5,A,9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.00,1,4637
...,...,...,...,...,...,...,...,...,...,...,...
9975,9976,I,80,sulfasalazine,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NCEXYHBECQHGNR-QZQOTICOSA-N,C(=O)(O)c1cc(ccc1O)\N=N\c1ccc(cc1)S(=O)(Nc1ccc...,-6.140000,0.00,1,12269
9976,9977,I,83,sulindac_form_II,InChI=1S/C20H17FO3S/c1-12-17(9-13-3-6-15(7-4-1...,MLKXDPUZXIRXEP-RQZCQDPDSA-N,CC1=C(CC(O)=O)c2cc(F)ccc2C\1=C\c1ccc(cc1)S(C)=O,-4.500000,0.41,2,12270
9977,9978,I,84,tetracaine,InChI=1S/C15H24N2O2/c1-4-5-10-16-14-8-6-13(7-9...,GKCBAIGFKIBETG-UHFFFAOYSA-N,C(c1ccc(cc1)NCCCC)(=O)OCCN(C)C,-3.010000,0.00,1,12271
9978,9979,I,85,tetracycline,InChI=1S/C22H24N2O8/c1-21(31)8-5-4-6-11(25)12(...,OFVLGDICTFRJMM-WESIUVDSSA-N,OC1=C(C(C2=C(O)[C@@](C(C(C(N)=O)=C(O)[C@H]3N(C...,-2.930000,0.00,1,12272


In [52]:
species.id.max()

4632

In [67]:
new_aqsol = pd.concat([matched_aqsol,unmatched_aqsol],axis=0)
new_aqsol.head()

,id,dataset,setnumber,name,inchi,inchikey,smiles,solubility,sd,ocurrences,species_id
5,6,A,10,vinyltoluene,"InChI=1S/C9H10/c1-3-9-6-4-5-8(2)7-9/h3-7H,1H2,2H3",JZHGRUMIRATHIU-UHFFFAOYSA-N,Cc1cccc(C=C)c1,-3.123150,0.000000,1,418
16,17,A,24,2-(4-chloro-2-methylphenoxy)propanoic acid,InChI=1S/C10H11ClO3/c1-6-5-8(11)3-4-9(6)14-7(2...,WNTGYJSOUMFZEP-UHFFFAOYSA-N,CC(Oc1ccc(Cl)cc1C)C(O)=O,-2.466031,0.060621,4,3058
18,19,A,26,1-chloro-3-(trifluoromethyl)benzene,"InChI=1S/C7H4ClF3/c8-6-3-1-2-5(4-6)7(9,10)11/h...",YTCGOUNVIAWCMG-UHFFFAOYSA-N,FC(F)(F)c1cccc(Cl)c1,-3.411514,0.000000,1,3696
21,22,A,29,1-methyl-4-(propan-2-ylidene)cyclohex-1-ene,"InChI=1S/C10H16/c1-8(2)10-6-4-9(3)5-7-10/h4H,5...",MOYAFQVGZZPNRA-UHFFFAOYSA-N,CC(C)=C1CCC(=CC1)C,-4.287343,0.131231,3,433
27,28,A,36,"(1S,5S)-6,6-dimethyl-2-methylidenebicyclo[3.1....","InChI=1S/C10H16/c1-7-4-5-8-6-9(7)10(8,2)3/h8-9...",WTARULDDTDQWMU-UHFFFAOYSA-N,CC1(C)C2CCC(=C)C1C2,-4.292313,1.247787,3,425


In [66]:
add_species = pd.DataFrame(columns=species.columns)
add_species

,id,iupac,formula,trivial,casrn,inchikey,subcat_id


In [69]:
add_species['id'] = unmatched_aqsol['species_id']
add_species['inchikey'] = unmatched_aqsol['inchikey']
add_species

,id,iupac,formula,trivial,casrn,inchikey,subcat_id
0,4633,NaN,NaN,NaN,NaN,SZEMGTQCPRNXEG-UHFFFAOYSA-M,NaN
1,4634,NaN,NaN,NaN,NaN,GPYLCFQEKPUWLD-UHFFFAOYSA-N,NaN
2,4635,NaN,NaN,NaN,NaN,AVPYQKSLYISFPO-UHFFFAOYSA-N,NaN
3,4636,NaN,NaN,NaN,NaN,XTUPUYCJWKHGSW-UHFFFAOYSA-L,NaN
4,4637,NaN,NaN,NaN,NaN,FAUAZXVRLVIARB-UHFFFAOYSA-N,NaN
...,...,...,...,...,...,...,...
9975,12269,NaN,NaN,NaN,NaN,NCEXYHBECQHGNR-QZQOTICOSA-N,NaN
9976,12270,NaN,NaN,NaN,NaN,MLKXDPUZXIRXEP-RQZCQDPDSA-N,NaN
9977,12271,NaN,NaN,NaN,NaN,GKCBAIGFKIBETG-UHFFFAOYSA-N,NaN
9978,12272,NaN,NaN,NaN,NaN,OFVLGDICTFRJMM-WESIUVDSSA-N,NaN


In [79]:
new_species = pd.concat([species,add_species],axis=1)
new_species['id'] = np.arange(1,new_species.shape[0]+1)
new_species = new_species.reset_index(drop=True)
new_species

,id,iupac,formula,trivial,casrn,inchikey,subcat_id,id,iupac,formula,trivial,casrn,inchikey,subcat_id
0,1,oxygen,O<sub>2</sub>,NaN,7782-44-7,MYMOFIZGZYHOMD-UHFFFAOYSA-N,1,1,NaN,NaN,NaN,NaN,SZEMGTQCPRNXEG-UHFFFAOYSA-M,NaN
1,2,ozone,O<sub>3</sub>,NaN,10028-15-6,CBENFWSGALASAD-UHFFFAOYSA-N,1,2,NaN,NaN,NaN,NaN,GPYLCFQEKPUWLD-UHFFFAOYSA-N,NaN
2,3,hydrogen atom,H,NaN,12385-13-6,YZCKVEUIGOORGS-UHFFFAOYSA-N,2,3,NaN,NaN,NaN,NaN,AVPYQKSLYISFPO-UHFFFAOYSA-N,NaN
3,4,hydrogen,H<sub>2</sub>,NaN,1333-74-0,UFHFLCQGNIYNRP-UHFFFAOYSA-N,2,4,NaN,NaN,NaN,NaN,XTUPUYCJWKHGSW-UHFFFAOYSA-L,NaN
4,5,deuterium,D<sub>2</sub>,NaN,7782-39-0,UFHFLCQGNIYNRP-VVKOMZTBSA-N,2,5,NaN,NaN,NaN,NaN,FAUAZXVRLVIARB-UHFFFAOYSA-N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,8652,NaN,NaN,NaN,NaN,NaN,NaN,8652,NaN,NaN,NaN,NaN,NCEXYHBECQHGNR-QZQOTICOSA-N,NaN
8652,8653,NaN,NaN,NaN,NaN,NaN,NaN,8653,NaN,NaN,NaN,NaN,MLKXDPUZXIRXEP-RQZCQDPDSA-N,NaN
8653,8654,NaN,NaN,NaN,NaN,NaN,NaN,8654,NaN,NaN,NaN,NaN,GKCBAIGFKIBETG-UHFFFAOYSA-N,NaN
8654,8655,NaN,NaN,NaN,NaN,NaN,NaN,8655,NaN,NaN,NaN,NaN,OFVLGDICTFRJMM-WESIUVDSSA-N,NaN


In [ ]:
# write a function that checks if species.id matches w inchikeys in both aqsol and hlc
def id_checker(db):
    pass